# 第7章　基本的な運用管理

## 7-1　データベースユーザの追加／削除／変更

### 7-1-1　ユーザの追加

ユーザには2種類ある。

・一般ユーザ<br>
・スーパーユーザ

スーパーユーザは、非常に強い権限を持つため<br>
通常はデータベース作成権限とユーザ作成権限を持つユーザの作成が推奨されている。

In [ ]:
# パスワード付きの一般ユーザを作成
createuser -P user1

In [ ]:
# user1がデータベース（examdb）にパスワード認証付きで接続する
psql -U user1 examdb

In [ ]:
# 登録されているユーザの一覧を出力し、user1が登録されていることを確認
examdb=> \du
---
Role nama | Member of
user1     | {}

### 7-1-2　ユーザの削除

In [ ]:
# user1を削除する
dropuser user1

# user1でexamdbに接続しようとするとエラーになる
psql -U user1 examdb

### 7-1-3　ユーザの変更

ALTERUSERで、ユーザの様々な設定を変更できる。

・SUPERUSER／NOUSUPERUSER：スーパユーザ権限の有無<br>
・CREATEDB／NOCREATEDB：データベース作成権限の有無<br>
・CREATEROLE／NOCREATEROLE：ユーザ作成権限の有無<br>
・LOGIN／NOLOGIN：ログイン権限の有無<br>
・PASSWORD 'パスワード'：パスワードの設定または変更<br>
・VALID UNTIL '日付'：パスワードの有効期限<br>

In [ ]:
# パスワードの設定または変更を行う
ALTER USER user1 WITH PASSWORD 'password';

In [ ]:
# psqlコマンドでも可能
\password user1

In [ ]:
# user1にスーパーユーザ権限を設定する
ALTER USER user1 WITH SUPERUSER;

# user1からスーパーユーザ権限を取り消す
ALTER USER user1 WITH NOSUPERUSER;

## 7-2　VACUUM、ANALYZE

### 7-2-1　VACUUM

PostgreSQLでは、MVCCを実装している。

・MVCC（Multi-Version-Concurrency-Control：多版型同時実行制御）：書き込みロックと読み込みロックが衝突しない仕組み

例えば、500円の商品データがある際に、Aさんが800円に更新するためのロックと、Bさんが読み込むためのロックが<br>
通常であれば衝突してどちらかが待たされるはずだが、MVCCは領域を確保して、2つの処理を同時に実行できるようにする。

しかし、不要領域が増えると様々な弊害が出てくるため、不要領域の回収をする必要がある。<br>
また、**データを削除する際**にも同様に**不要領域が発生**する。そのための処理が、**VACUUM**である。

・VACUUM：不要領域を**回収**する（物理的な削除ではなく、再利用可能なマークをつける）、**同時にSQLの実行もできる**<br>
・VACUUM FULL：テーブル内容を新しいファイルに書き換えて、**物理的に削除**、排他ロックをするので**SQLの同時実行はできない**

VACUUM FULLは、大量のデータ削除によって不要領域が大半を占める際に利用すべきであり、定期的には実行しない。

In [ ]:
# テーブル（tab1）に対してVACUUMを実行する
VACUUM tab1;
---
VACUUM

In [ ]:
# データベースを指定しない場合、データベース内のすべてのテーブルに対してVACUUMを実行する
VACUUM;

In [ ]:
# VERBOSEオプションで、実際に回収した不要領域を知ることができる
VACUUM VERBOSE tab1;

VERBOSEログは標準ログに出力されるので、注意すること。

### 7-2-2　ANALYZE

データベースへのSQLの実行は、テーブルに実際にどのようにデータが格納されているかという統計情報に基づいて行われる。<br>
この統計情報を更新するのが、ANALYZEである。テーブル名を省略すると、すべてのテーブルが対象となる。

In [ ]:
# テーブル（tab1）に対して、ANALYZEで統計情報を更新する
ANALYZE tab1;

### 7-2-3　VACUUM ANALYZE

不要領域を回収するVACUUMと、統計情報を更新するANALYZEをまとめて実行することもできる。

In [ ]:
VACUUM ANALYZE tab1;

## 7-3　自動バキューム

PostgreSQLには、自動バキューム（autovacuum：オートバキューム）という機能がある。<br>
これは、テーブル内の不要領域の割合が多くなったテーブルに、自動で**VACUUM**と**ANALYZE**を実行する。<br>

**postgresql.conf**ファイルに設定があり、デフォルトで有効（autovacuum = on）になっている。

## 7-4　システム情報取得関数

運用管理では、操作の前後でシステムの情報を確認することがある。<br>
このとき、システム情報関数を利用する。

### 7-4-1　version（）関数

サーバで稼働しているPostgreSQLのバージョンは、version()関数で確認できる。

In [ ]:
# psqlコマンドのバージョンを確認する
psql

# PostgreSQLのバージョンを確認する
SELECT version()

### 7-4-2　current_database（）関数

現在接続しているデータベースは、current_database()関数で確認できる。

In [ ]:
SELECT current_database();

### 7-4-3　current_userとuser

current_userとuserは、どちらも現在のユーザを確認できる。<br>
上の関数と違い、**どちらも最後に()をつけない**ことに注意する。

In [ ]:
SELECT current_user;

SELECT user;

## 7-5　情報スキーマ、システムカタログ

データベースクラスタ全体にかかわる各種の情報は<br>
**情報スキーマ**と**システムカタログ**（pg_roles、pg_authid）に格納されている。

### 7-5-1　情報スキーマ（information_schema）

情報スキーマ：データベース内のテーブルや列、ユーザなどの様々な情報における**定義情報**が格納されている<br>
システムカタログ：**PosgreSQL固有の情報**や、テータベースに関する内部情報が格納されている

In [ ]:
# 情報スキーマから現在有効なユーザ（enabled_roles）の情報を表示する
SELECT * FROM information_schema.enabled_roles;

情報スキーマのtablesビュー（information_schema.tables）では、**データベースに登録されている全てのテーブル**を調べることができる。

In [ ]:
SELECT * FROM information_schema.tables;

### 7-5-2　システムカタログ（pg_rolesビュー、pg_authidカタログ）

システムカタログは、データベースに関する内部情報を格納したテーブルである。<br>
PostgreSQLのシステムカタログは、**pg_catalog**という名前のスキーマで定義されている。

In [ ]:
# psqlコマンド（\du）で表示されるようなユーザに関する情報を参照する
SELECT * FROM pg_roles WHERE rolcanlogin IS TRUE;

pg_rolesビューは、pg_authidカタログのビューである。

pg_authidカタログは、MD5でハッシュ化されたパスワードをrolpasswordに含んでおり<br>
pg_rolesビューでは見えないようになっている。

※ハッシュ化：パスワードを他の適当な文字に置き換えること（暗号化）

In [ ]:
# スキーマも含めて指定する
SELECT * FROM pg_catalog.pg_roles;

### 7-5-3　その他のシステムカタログ

pg_settingビューでは、**SHOW文を扱うようなパラメータ設定を参照**できる。

In [ ]:
SELECT * FROM pg_settings;

pg_databaseカタログには、データベースの情報が格納されている。<br>
尚、pg_databaseカタログは、データベースごとに存在するものではなく、**データベースクラスタごと**に1つ存在する。

In [ ]:
SELECT * FROM pg_database;

schemanameがpublicと一致する行をpg_tablesビューを使って検索すると<br>
ユーザが作成したテーブルを調べることができる。

In [ ]:
SELECT * FROM pg_tables WHWRE schemaname='public';

情報スキーマとシステムカタログをまとめると、以下の通りになる

・information_schema.enabled_roles：現在有効なユーザの情報を表示する<br>
・information_schema.tables：**現在接続中の**データベースに含まれている全てのテーブルを調べる<br>

・pg_authid：MD5でハッシュ化されたパスワードをrolpasswordに含むカタログ<br>
・pg_roles：pg_authidのビュー<br>
・pg_setting：SHOW文を扱うようなパラメータ設定を参照できる<br>
・pg_database：データベースの情報を表示する、データベースクラスタごとに1つ存在する<br>
・pg_tables：データベース内のテーブルに関する情報（スキーマ名、テーブル名、テーブル所有者、インデックスの有無）を表示する

## 7-6　テーブル単位の権限、GRANT 文とREVOKE 文

テーブル単位で権限を設定するにはGRANT文、権限を取り消すにはREVOKE文を使用する。

### 7-6-1　GRANT 文

In [ ]:
# user1が作成したテーブル（tab1）を、誰からでも参照可能に設定する
GRANT SELECT ON tab1 TO PUBLIC;

# 結果を見る
\dp
---
user1=arwdDxt/user1+
# user1が作成したテーブルに対して、すべてのユーザにSELECT権限が設定されている（左辺なしは、全てのユーザを表す）
=r/user1

In [ ]:
GRANT SELECT, UPDATE ON tab2 TO user3;

\dp
---
user1=arwdDxt/user1+
# user1が作成したテーブルに対して、user3にrw権限が設定されている
user3=rw/user1

UPDATE権限だけの設定だと、参照を伴うような更新処理はエラーとなってしまう。<br>
そのため、UPDATEを設定する際には、SELECT権限も併せて設定すると良い。

### 7-6-2　REVOKE 文

テーブルに設定されたアクセス権限を取り消すには、REVOKE権限を使用する。

In [ ]:
# まず権限を確認する
\dp
---
user3=rw/user1

# 権限を取り消す
REVOKE ALL ON tab2 FROM user3;

\dp tab2
---
# user3の権限がなくなっている
user1=arwdDxt/user1

In [ ]:
# 全てのユーザは、全ての権限を持っている
\dp tab3
---
user1=arwdDxt+
=arwdDxt/user1

REVOKE INSERT, UPDATE, DELETE, TRUNCATE, REFERENCES, TRIGGER ON tab3 FROM PUBLIC;

\dp tab3
---
# 参照権限のみを残っている
user1=arwdDxt+
=r/user1

### 7-6-3　\dp または \z（テーブル権限の確認）

psqlコマンドの**\dp**または**z**で、各テーブルのアクセス権限を確認できる。<br>
引数としてテーブルを指定しなかった場合には、テーブル内の全てのテーブルについて権限の情報を表示する。

In [ ]:
\dp

\dp tab1

権限の見方を、下記に示す。

・ユーザ名=xxxx：ユーザに設定された権限<br>
・=xxxx：すべてのユーザに設定された権限<br>
・r：SELECT（読み取り：r）<br>
・w：UPDATE（書き込み：w）<br>
・a：INDERT（追加：append）<br>
・d：DELETE<br>
・D：TRANCATE（テーブルそのものではなく、テーブルの中身を空にする）<br>
・x：REFFERENCES（外部キー制約の作成）<br>
・t：TRIGGER（トリガの作成）<br>
・arwdDxt：すべての権限<br>
・/yyyy：この権限を設定したユーザ